# Import

In [2]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
import itertools
from ast import literal_eval

# One card

## Data structure

In [3]:
collected = {
    "stone":0,
    "animal":0,
    "plant":0,
}

content = {
    "color" : "blue",
    
    "number" : 46,
    "night" : False,
    "clue" : True,
    
    "ressources":[],
    
    "requirements":[
        collected["stone"]>=2,
        collected["animal"]>=1,
        ],
    "reward":10,
}

content.keys()

dict_keys(['color', 'number', 'night', 'clue', 'ressources', 'requirements', 'reward'])

## Class

In [4]:
class Sanctuary():
    def __init__(self, color, night=False, clue=False, ressources=[], reward_type='fix', reward_value=0):
        self.color = color
        
        self.night=night
        self.clue=clue
        
        self.ressources=ressources
        self.reward_type=reward_type
        self.reward_value=reward_value
        
    def compute_reward(self, collected):
        if isinstance(self.reward_type, list):
            value = 0
            for type in self.reward_type:
                value += collected[type]*self.reward_value 
            return value
                
        elif self.reward_type == 'fix':
            return self.reward_value
        elif self.reward_type == 'multi':
            return collected[['blue', 'green', 'red', 'yellow']].min()*self.reward_value
        else:
            return collected[self.reward_type]*self.reward_value 
        
    def get_reward(self, collected):
        return self.compute_reward(collected)
    
    def get_ressources(self, collected):
        collected[self.color] += 1
        
        if self.night:
            collected["night"] += 1
            
        if self.clue:
            collected["clue"] += 1
            
        for ressource in self.ressources:
            collected[ressource] += 1
            
        collected['score'] += self.get_reward(collected)


class Card(Sanctuary):
    def __init__(self, color, number, night=False, clue=False, ressources=[], requirements=[], reward_type='fix', reward_value=0):
        super().__init__(color=color, night=night, clue=clue, ressources=ressources, reward_type=reward_type, reward_value=reward_value)
        
        self.number = number
        self.requirements = pd.Series(requirements).value_counts().to_dict()
        
    def get_reward(self, collected):
        if all([collected[type]>=number for type, number in self.requirements.items()]):
            return self.compute_reward(collected)
        else :
            return 0
        
    
            


collected = pd.Series({
    "stone":2,
    "animal":2,
    "plant":1,
    
    "clue":0,
    "night":0,
    
    "blue":2,
    "green":1,
    "red":3,
    "yellow":2,
    
    "score":0,
})

card_25 = Card(color="yellow", number=25, night=True, clue=False, ressources=[], requirements=[], reward_type=['yellow', 'green'],reward_value=1)
card_25.get_ressources(collected)
display(collected)
card_34 = Card(color="green", number=34, night=True, clue=False, ressources=["animal"], requirements=["stone", "stone"], reward_type='animal',reward_value=3)
card_34.get_ressources(collected)
display(collected)
card_35 = Card(color="yellow", number=35, night=True, clue=False, ressources=["animal"], requirements=[], reward_type='multi',reward_value=10)
card_35.get_ressources(collected)
display(collected)
card_46 = Card(color="blue", number=46, night=False, clue=True, ressources=[], requirements=["stone", "stone", "animal"], reward_type='fix',reward_value=10)
card_46.get_ressources(collected)
display(collected)


C:\Users\mathi\AppData\Local\Temp\ipykernel_18292\3787086781.py:49: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.requirements = pd.Series(requirements).value_counts().to_dict()


stone     2
animal    2
plant     1
clue      0
night     1
blue      2
green     1
red       3
yellow    3
score     4
dtype: int64

stone      2
animal     3
plant      1
clue       0
night      2
blue       2
green      2
red        3
yellow     3
score     13
dtype: int64

C:\Users\mathi\AppData\Local\Temp\ipykernel_18292\3787086781.py:49: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.requirements = pd.Series(requirements).value_counts().to_dict()


stone      2
animal     4
plant      1
clue       0
night      3
blue       2
green      2
red        3
yellow     4
score     33
dtype: int64

stone      2
animal     4
plant      1
clue       1
night      3
blue       3
green      2
red        3
yellow     4
score     43
dtype: int64

# All Cards

In [5]:
data = [
    {'color':"red", 'number':1, 'night':False, 'clue':False, 'ressources':["animal", "stone"], 'requirements':[], 'reward':('fix',0)},
    {'color':"blue", 'number':2, 'night':False, 'clue':False, 'ressources':["stone", "stone"], 'requirements':[], 'reward':('fix',0)},
]


In [6]:
df = pd.read_csv('all_cards.csv')
df = df.dropna()
df['reward_value']=df['reward_value'].astype(int)

display(df)

result = df.apply(lambda x : Card(color=x['color'], number=x['number'], night=x['night'], clue=x['clue'], ressources=x['ressources'], 
                                  requirements=x['requirements'], reward_type=x['reward_type'],reward_value=x['reward_value']), axis=1)
    
result.apply(lambda x : x.color).value_counts()

,color,number,night,clue,ressources,requirements,reward_type,reward_value
0,red,1,False,False,"['animal', 'stone']",[],fix,0
1,blue,2,False,False,"['stone', 'stone']",[],fix,0
2,green,3,False,False,[],[],fix,4
3,red,4,False,False,"['plant', 'stone']",[],fix,0
4,green,5,False,False,['animal'],[],fix,2
...,...,...,...,...,...,...,...,...
63,blue,64,False,True,[],"['stone','stone','plant','plant']",fix,18
64,yellow,65,False,True,[],"['plant','plant','plant']","['yellow','green']",3
65,blue,66,False,False,[],"['stone','stone','stone','stone']",fix,20
66,green,67,False,True,[],"['animal','animal','plant','plant']",fix,19


red       17
blue      17
green     17
yellow    17
dtype: int64

# All Sanctuaries

In [7]:
data = [
    {'color':"red", 'night':False, 'clue':False, 'ressources':[], 'reward_type':'red', 'reward_value':1},
    {'color':"grey", 'night':False, 'clue':False, 'ressources':['stone'], 'reward_type':'night', 'reward_value':1},
]

In [8]:
df = pd.read_csv('all_sanctuaries.csv')
df = df.dropna()
df['reward_value']=df['reward_value'].astype(int)

display(df)

result = df.apply(lambda x : Sanctuary(color=x['color'], night=x['night'], clue=x['clue'], ressources=x['ressources'], 
                                reward_type=x['reward_type'],reward_value=x['reward_value']), axis=1)
    
result.apply(lambda x : x.color).value_counts()

,color,night,clue,ressources,reward_type,reward_value
0,red,False,False,[],red,1
1,grey,False,False,['stone'],night,1
2,red,True,False,[],fix,0
3,grey,False,False,['animal'],animal,1
4,grey,False,False,[],stone,2
5,grey,False,False,[],clue,2
6,green,False,True,[],fix,0
7,grey,False,True,['plant'],fix,0
8,blue,False,False,[],blue,1
9,red,False,False,['stone'],fix,0


grey      25
red        5
green      5
blue       5
yellow     5
dtype: int64

# Exploration

## Loading

In [9]:
card_df = pd.read_csv('all_cards.csv')
for col in ['ressources', 'requirements','reward_type']:
    card_df[col]=card_df[col].apply(lambda x : x if x[0]!='[' else literal_eval(x))
card_df = card_df.set_index('number', drop=False)
card_df['instance'] = card_df.apply(lambda x : Card(color=x['color'], number=x['number'], night=x['night'], clue=x['clue'], ressources=x['ressources'], 
                                  requirements=x['requirements'], reward_type=x['reward_type'],reward_value=x['reward_value']), axis=1)

sanctuary_df = pd.read_csv('all_sanctuaries.csv')
for col in ['ressources','reward_type']:
    sanctuary_df[col]=sanctuary_df[col].apply(lambda x : x if x[0]!='[' else literal_eval(x))
sanctuary_df['instance'] = sanctuary_df.apply(lambda x : Sanctuary(color=x['color'], night=x['night'], clue=x['clue'], ressources=x['ressources'], 
                                reward_type=x['reward_type'],reward_value=x['reward_value']), axis=1)

C:\Users\mathi\AppData\Local\Temp\ipykernel_18292\3787086781.py:49: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.requirements = pd.Series(requirements).value_counts().to_dict()
C:\Users\mathi\AppData\Local\Temp\ipykernel_18292\3787086781.py:49: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.requirements = pd.Series(requirements).value_counts().to_dict()
C:\Users\mathi\AppData\Local\Temp\ipykernel_18292\3787086781.py:49: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.requirements = pd.Series(requirements).value_counts().to_dict()
C:\Users\mathi\AppData\Local\Temp\ipykernel_18292\3787086781.py:49: FutureWarning: The default dtype

## Permutations

In [22]:
from collections import defaultdict
# itertools.permutations(card_df['number'], r=8)

sequence = random.choices(card_df['number'], k=8)

# sequence = [62, 17, 48, 66, 12, 3, 26, 12]
r_sequence = reversed(sequence)

def compute_score(sequence):
    collected = pd.Series({
    "stone":0,
    "animal":0,
    "plant":0,
    
    "clue":0,
    "night":0,
    
    "blue":0,
    "green":0,
    "red":0,
    "yellow":0,
    
    "score":0,
    })

    for i in sequence:
        card_df.loc[i]['instance'].get_ressources(collected)
        
    return collected['score']
    
compute_score(r_sequence)

6

# All Combinations

In [11]:
# all_combinations = [elem for elem in itertools.permutations(card_df['number'], r=8)]

In [21]:



mydf['sequence'].apply(compute_score)
    

0    34
1    30
2    30
3    34
4    32
5    30
6    36
7    33
8    33
9    34
Name: sequence, dtype: int64